In [ ]:
import math as mt
import numpy as np
from Plots.Velocity_plot import Graph_V,plot_V
import matplotlib.pyplot as plt

dia=0.0154
length=2
Grid_points=200
H=dia/100
W=length/4
dx=length/Grid_points
r_effective=np.zeros((2*Grid_points+1))

for i in range(0,2*Grid_points+1):
    z= (length/2)-(i*dx/2)
    r_effective[i]=(dia/2)+(H*mt.exp(-z**2/(2*(W**2))))

graph_v=Graph_V()
x2=np.arange(len(r_effective))
plot_V(r_effective,x2,graph_v)

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

# Your data
x = np.array([0, 0.010827067669172935, 0.02165413533834587, 0.035187969924812046, 0.048721804511278194, 0.060902255639097735, 0.0730827067669173, 0.0825563909774436, 0.0893233082706767, 0.09744360902255639, 0.10421052631578948, 0.11097744360902256, 0.1177443609022556, 0.12586466165413535, 0.13263157894736843, 0.13804511278195492, 0.14616541353383455, 0.15699248120300752, 0.16646616541353382, 0.1786466165413534, 0.1894736842105263, 0.20030075187969926, 0.20977443609022556, 0.22195488721804513, 0.23007518796992482, 0.23954887218045112, 0.24766917293233082, 0.2557894736842105, 0.2666165413533835, 0.2760902255639098, 0.2855639097744361, 0.2923308270676692, 0.30045112781954886, 0.30992481203007516, 0.31939849624060146, 0.3275187969924812, 0.3369924812030075, 0.3464661654135339, 0.35458646616541356, 0.36406015037593986, 0.3735338345864663, 0.38300751879699246, 0.39248120300751876, 0.40601503759398505, 0.4195488721804511, 0.43443609022556384, 0.44661654135338347, 0.4615037593984962, 0.47233082706766927, 0.487218045112782, 0.5007518796992482, 0.5142857142857142, 0.5291729323308271, 0.5413533834586466, 0.5535338345864661, 0.5684210526315789, 0.5792481203007519, 0.5914285714285715, 0.6063157894736843, 0.6171428571428571, 0.6320300751879699, 0.6442105263157895, 0.6590977443609023, 0.6712781954887218, 0.684812030075188, 0.695639097744361, 0.709172932330827, 0.7227067669172933, 0.7362406015037594, 0.7497744360902256, 0.7619548872180452, 0.7754887218045113, 0.7876691729323309, 0.8025563909774437])
y = np.array([0, 0.0786026200873362, 0.1179039301310052, 0.1965065502183414, 0.3930131004366828, 0.6681222707423586, 0.9432314410480345, 1.3362445414847173, 1.6899563318777293, 2.082969432314412, 2.5152838427947604, 2.8689956331877724, 3.2620087336244534, 3.6550218340611345, 3.969432314410481, 4.323144104803495, 4.598253275109171, 4.794759825327512, 4.951965065502183, 4.951965065502183, 4.794759825327512, 4.598253275109171, 4.323144104803495, 3.969432314410481, 3.7336244541484724, 3.458515283842795, 3.222707423580786, 2.9475982532751104, 2.5938864628820966, 2.2794759825327517, 1.9650655021834051, 1.650655021834062, 1.4541484716157207, 1.179039301310043, 0.9432314410480345, 0.6681222707423586, 0.3144104803493448, 0.0786026200873362, -0.1572052401746724, -0.3930131004366828, -0.6288209606986896, -0.7467248908296966, -0.9039301310043673, -0.9432314410480327, -0.9432314410480327, -0.8646288209606983, -0.8253275109170328, -0.6288209606986896, -0.5502183406113534, -0.43231441048034824, -0.3144104803493448, -0.23580786026200862, -0.1572052401746724, -0.0786026200873362, 0, 0, 0.0786026200873362, 0.0786026200873362, 0.1179039301310052, 0.1179039301310052, 0.1179039301310052, 0.1179039301310052, 0.0786026200873362, 0.1179039301310052, 0.0786026200873362, 0.0786026200873362, 0.0786026200873362, 0.0786026200873362, 0, -0.0786026200873362, -0.0786026200873362, -0.0786026200873362, -0.0786026200873362, -0.1179039301310052])

# Fit a spline to the data
spline = UnivariateSpline(x, y, s=0)

# Plot the original data
plt.scatter(x, y, label='Original data')

# Plot the fitted spline
xs = np.linspace(min(x), max(x), 1000)
ys = spline(xs)
plt.plot(xs, ys, label='Spline fit')

plt.xlabel('Time (sec)')
plt.ylabel('Flow rate (lit/min)')
plt.legend()
plt.show()


In [ ]:
# Get the coefficients and knots
coefficients = spline.get_coeffs()
knots = spline.get_knots()

# Print the coefficients and knots
#print("Coefficients:", coefficients)
#print("Knots:", knots)

# Use the spline object to get a y value for a specific x
x_value = 0.01
y_value = spline(x_value)
print(f"The y value at x={x_value} is {y_value}")

In [ ]:
import numpy as np
import os
import time 
import math as mt
import matplotlib.pyplot as plt
import matlab.engine
from Modules.Momentum import Momentum
from Modules.Pressure import Pressure_adjust
from Update.Pressure_update import add
from Modules.convergence import convergence
from Modules.Pressure import Pressure_adjust
#from Plots.Pressure_plot import plot_pressure
from Plots.Plot import Graph_PV,plot
from Plots.Velocity_plot import Graph_V,plot_V
from Plots.Pressure_plot import  Graph_P,plot_P 
from Plots.Area_profile import Graph_Area_profile,plot_Area
from Modules.Correct_velocity import Correct_velocity
from Modules.Correct_pressure import Correct_pressure
from Area.linear_area import Linear_area_profile, Const_area_profile,Area
eng=matlab.engine.start_matlab()
eng.addpath(r'E:\BTP\Linux\Fluid_flow\Unsteady-Fluid-Flow-Model\Matlab')

In [ ]:

"""
const paramers through-out the analysis
All units are taken in SI system (Kilogram,Meter,Second)

"""
Total_time=1
#dt = 0.001                              # Time step size                 # Number of time steps

rho = 996.550                          # Density kg/m3
g = 9.8                                # Gravitational acceleration m/s2
Grid_points=500               #Total number of grid points (Cell centres including extra cell at the end)
Inletmassflux=900                      #kg/m2.s
P_atm=1                                #atm
dia=0.0154                             #m
d_vis=0.000854                         #Ns/m2
p_exit= P_atm* 101325                  #N/m2
A= (mt.pi)*(dia**2)/4                  #m2
u_inlet= Inletmassflux/(rho)           #m/s
length=2                               #m
dx = length/Grid_points 
dt=0.0040             # Spatial grid size
#n = int(Total_time/dt)
n=4 
CFL=u_inlet*dt/dx
minimum_dt=dx/u_inlet     #<0.0044


"""
Variables evolve-with time
graph=Graph_PV()
a1=np.arange(0,len(A_n))
a2=np.arange(0,len(r_x))
plot(A_n,r_x,a1,a2,graph)
plt.pause(0.5)
plt.show()
graph=Graph_Area_profile()
x=np.arange(0,len(r_x1))
plot_Area(x,r_x1,r_x2,A_n,p_s,graph)
plt.pause(0.5)
plt.show()
"""
# Initialize arrays for velocity (u) and pressure (p) at staggered grid locations
#A_n = np.full((2 * Grid_points + 1), A)                # Area at n_th step
#perimeter= (mt.pi)*dia                                 #perimeter at n_th step 
#p_s=np.full((2 * Grid_points + 1), perimeter)
l1=length/4
l2=3*length/4
r_x1,r_x2,A_n,p_s=Area(dia,Grid_points,length,l1,l2)   #Area and perimeter at n_th step     
dp= 128*d_vis*length*A*u_inlet/((mt.pi)*dia**4)        #pressure change for end points(Change the length to get the desired pressure drop)
print(dp,"dp")


"""
Initialize the velocity 
"""
u_n = np.zeros(Grid_points+1)  
u_n[0]=u_inlet
for i in range(1,len(u_n)):
    u_n[i]=0.01    


x_value=0.2
y_value = spline(x_value)
u_in=(((y_value*10**-3)/60)/(A))
print(u_in,"u_in")
print(u_inlet,"u_inlet")




# Main Algo
def unsteady_1D_flow(A_n,u_n,p_s,Grid_points,rho,dx,dt,d_vis,n,u_inlet,p_exit):
    p_star = np.zeros(Grid_points)
    u_star = np.zeros(Grid_points+1) 
    x1=np.arange(0,len(p_star))
    x2=np.arange(0,len(u_star))

    """
    Initialize the pressure 
    """
    p_star[len(p_star)-1]=p_exit
    for i in range(len(p_star)-2,-1,-1):
        #p_star[i]=p_star[i+1]+((dp*dx)/length)
        p_star[i]=p_star[i+1]
    u_star=u_n
    #Area=np.full((2 * Grid_points + 1), A)
    Area=np.ones((2 * Grid_points + 1))
    Area=A_n
    for t in range(0,1):
        x_value=0.1
        y_value = spline(x_value)
        u_in=(((y_value*10**-3)/60)/(Area[0]))   
        u_n[0]=u_in
        for i in range(1,len(u_n)):
           u_n[i]=0.001
        start_time=time.time()
        i=0
        u_star=Momentum(p_star,u_n,u_star,Grid_points,rho,dt,dx,d_vis,A_n,Area,p_s,u_inlet,p_exit) 
        while True:
            i=i+1
            print(i)
            x2_list=x2.tolist()
            u_star_list=u_star.tolist() 
            eng_x2 = matlab.double(x2_list)
            eng_u = matlab.double(u_star_list)
            eng_x1 = matlab.double(x1.tolist())
            eng_p = matlab.double(p_star.tolist())
            eng.graph_V(eng_x2, eng_u, nargout=0)
            eng.graph_P(eng_x1, eng_p, nargout=0)
            converge=convergence(u_star,Grid_points,rho,Area,A_n,dx,dt)
            print(converge,"converge")
            if converge<10**-2:
                break                                                                         
            p_add=Pressure_adjust(u_star,Grid_points,u_n,Area,A_n,p_s,rho,dx,dt,d_vis,u_inlet,p_exit)
            #relaxation_factor=1
            #p_add=relaxation_factor*p_add
            #p_star=add(p_add,p_star,relaxation_factor)
            u_star=Correct_velocity(u_star,p_add,Grid_points,rho,dt,dx,d_vis,A_n,Area,u_n,p_s)
            p_star=Correct_pressure(p_star,u_n,u_star,Grid_points,rho,dt,dx,d_vis,A_n,Area,p_s)
            #relaxation_factor=10**-5
            #p_star=add(p_add,p_star,relaxation_factor)
            #u_star=Momentum(p_star,u_n,u_star,Grid_points,rho,dt,dx,d_vis,A_n,Area,p_s,u_inlet,p_exit) 
            #u_star=u_update
            elasp_time = time.time() - start_time
            if elasp_time>100:
                break
            # Save the current figure with a unique file name
            # # Unique file name based on iteration
            #plt.savefig(file_name)
        
            #plt.clf()  # Clear the current figure to prepare for the next iteration  
        u_n=u_star
    plt.show()    
    return u_star,p_star

"""
Final result 
"""

#velocity, Pressure =unsteady_1D_flow(A_n,u_n,p_s,Grid_points,rho,dx,dt,d_vis,n,u_inlet,p_exit)


In [ ]:
x_value=0.35
y_value = spline(x_value)
u_in=(((y_value*10**-3)/60)/(A))
print(u_in,"u_in")
print(u_inlet,"u_inlet")


In [ ]:
from Boundary_conditions.velocity_p_time import velocity
u_in1=velocity(A,0)
print(u_in1,"u_in")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Make data

x = np.ones(100)
y = 1
z = np.ones(100)
# Plot the surface
ax.plot(x,z,y)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have x and y arrays of the same shape
x = np.linspace(0, 10, 100)
z = np.linspace(0, 5, 100)
X,Z=np.meshgrid(x,z)
# Create a new figure
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Plot four lines along the z-axis
y_values = np.linspace(0,5,100)  # Example z-values for the lines
for y in y_values:
    ax.plot(X, y*np.ones_like(X),Z)  # z*np.ones_like(x) creates an array of the same shape as x filled with z values

# Set labels for each axis
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# Display the plot
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have x and z arrays of the same shape
x = np.linspace(0, 10, 100)
z = np.linspace(0, 5, 100)
X,Z=np.meshgrid(x,z)
# Create a new figure
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Plot lines along the Y-axis
y_values = np.linspace(0,5,100)  # Example y-values for the lines
for y in y_values:
    ax.plot(X, y*np.ones_like(X),Z)  # y*np.ones_like(X) creates an array of the same shape as X filled with y values

# Set labels for each axis
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# Display the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have x and z arrays of the same shape
x = np.linspace(0, 10, 100)
z = np.linspace(0, 5, 100)


# Create a new figure
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Plot lines along the Y-axis
y_values = np.linspace(0,5,100)  # Example y-values for the lines
for y in y_values:
    Y = y*np.ones_like(x)  # Create a 2D array for Y
    ax.plot(x, Y, z)

# Set labels for each axis
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# Display the plot
plt.show()


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming you have x and z arrays of the same shape
x = np.linspace(0, 10, 100)
z = np.linspace(0, 5, 100)

# Create a new figure
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Plot lines along the Y-axis
y_values = np.linspace(0,5,10)  # Example y-values for the lines
for i, y in enumerate(y_values):
    Y = y*np.ones_like(x)  # Create a 2D array for Y
    ax.plot(x, Y, z, color=plt.cm.viridis(0.5), linewidth=2)

# Set labels for each axis
ax.set_xlabel('X-axis', fontsize=12)
ax.set_ylabel('Y-axis', fontsize=12)
ax.set_zlabel('Z-axis', fontsize=12)

# Set title
ax.set_title('3D Line Plot', fontsize=16)

# Display the plot
plt.show()


In [83]:
i=130%10
print(i)

0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define 1D arrays for x and y coordinates
x = np.linspace(-5, 5, 10)
y = np.linspace(-5, 5, 10)

# Creating a meshgrid from x and y
X, Y = np.meshgrid(x, y)

# Display the resulting meshgrid
print("X-coordinates:")
print(X)
print("Y-coordinates:")
print(Y)

# Plotting the grid points
plt.scatter(X, Y)
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Meshgrid')
plt.show()
